In [1]:
!pip install opendatasets
import opendatasets as od
dataset_url = 'https://www.kaggle.com/datasets/jangedoo/utkface-new'
od.download(dataset_url)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: abdelrhmanenait
Your Kaggle Key: ··········


100%|██████████| 331M/331M [00:02<00:00, 130MB/s]


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Load Dataset

In [1]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from PIL import Image
import cv2 as cv
from sklearn.model_selection import train_test_split
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [3]:
labels=[]
dataset = []
path = './utkface-new/UTKFace'
for filename in tqdm(os.listdir(path)[0:8000]):
  parts=filename.split('_')
  image = Image.open(os.path.join(path,filename))
  image = image.convert("RGB")
  input_tensor = transform(image)
  dataset.append(input_tensor)
  labels.append(int(parts[1]))
 
# dataset = np.array(dataset)
labels = np.array(labels)

# print(dataset.shape)
# _,cou=np.unique(labels, return_counts=True)
# cou

100%|██████████| 8000/8000 [00:37<00:00, 214.74it/s]


In [4]:
# dt = torch.load('./drive/MyDrive/MobileNetV3_Gender/dataset/dataset_6000_v2.pt')
# dt.extend(dataset)
# torch.save(dataset,'./drive/MyDrive/MobileNetV3_Gender/dataset/dataset_18k_23k.pt')
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [4]:
dataset = np.array(dataset)
from torchvision import models
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device object

<ipython-input-4-b1fcb0637249>:1: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  dataset = np.array(dataset)
<ipython-input-4-b1fcb0637249>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dataset = np.array(dataset)


# RESNET18

In [5]:
from torch.utils.data import Dataset, DataLoader

# Define a custom dataset class
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]




In [5]:
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2) # binary classification (num_of_class == 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
path = '/content/drive/MyDrive/MobileNetV3_Gender/model/model_resnet18_v5.pt'
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2) # binary classification (num_of_class == 2)
model.to(device)
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['model_state_dict'])
criterion = nn.CrossEntropyLoss()
criterion.load_state_dict(checkpoint['criterion_state_dict'])
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
num_epochs = 2
batch_size = 64
learning_rate = 0.001

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.33, random_state=112)
X_train = DataLoader(MyDataset(X_train), batch_size=batch_size, shuffle=False)
X_test = DataLoader(MyDataset(X_test), batch_size=batch_size, shuffle=False)
y_train = DataLoader(MyDataset(y_train), batch_size=batch_size, shuffle=False)
y_test = DataLoader(MyDataset(y_test), batch_size=batch_size, shuffle=False)

for epoch in range(num_epochs):
    """ Training Phase """
    model.train()

    running_loss = 0.
    running_corrects = 0

    # load a batch data of images
    for inputs, label in tqdm(zip(X_train, y_train)):
        inputs = inputs.to(device)
        label = label.to(device)

        # forward inputs and get output
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, label)

        # get loss value and update the network weights
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == label.data)
        
    epoch_loss = running_loss / len(y_train.dataset)
    epoch_acc = running_corrects / len(y_train.dataset) * 100.
    print('[Train #{}] Loss: {:.4f} Acc: {:.4f}%'.format(epoch, epoch_loss, epoch_acc))

    """ Validation Phase """
    model.eval()

    with torch.no_grad():
        running_loss = 0.
        running_corrects = 0

        for inputs, label in tqdm(zip(X_test,y_test)):
            inputs = inputs.to(device)
            label = label.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, label)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == label.data)

        epoch_loss = running_loss / len(y_test.dataset)
        epoch_acc = running_corrects / len(y_test.dataset) * 100.
        print('[Validation #{}] Loss: {:.4f} Acc: {:.4f}%'.format(epoch, epoch_loss, epoch_acc))

84it [00:17,  4.73it/s]


[Train #0] Loss: 0.1409 Acc: 95.1306%


42it [00:03, 13.10it/s]


[Validation #0] Loss: 0.1283 Acc: 95.4545%


84it [00:16,  5.00it/s]


[Train #1] Loss: 0.0370 Acc: 99.3097%


42it [00:03, 11.54it/s]

[Validation #1] Loss: 0.1286 Acc: 95.5682%


In [10]:
path = '/content/drive/MyDrive/MobileNetV3_Gender/model/model_resnet18_v6.pt'
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'criterion_state_dict': criterion.state_dict(),
}, path)

In [9]:
data = DataLoader(MyDataset(dataset), batch_size=1, shuffle=False)
y = DataLoader(MyDataset(labels), batch_size=1, shuffle=False)
model.eval()
for epoch in [1]:
  with torch.no_grad():
      running_loss = 0.
      running_corrects = 0

      for inputs, label in tqdm(zip(data,y)):
          inputs = inputs.to(device)
          label = label.to(device)

          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, label)

          running_loss += loss.item() * inputs.size(0)
          running_corrects += torch.sum(preds == label.data)

      epoch_loss = running_loss / len(y.dataset)
      epoch_acc = running_corrects / len(y.dataset) * 100.
      print('[Validation #{}] Loss: {:.4f} Acc: {:.4f}%'.format(epoch, epoch_loss, epoch_acc))

8000it [00:28, 283.36it/s]

[Validation #1] Loss: 0.0940 Acc: 96.5625%


# MobileNet3

In [7]:
from torch.utils.data import Dataset, DataLoader

# Define a custom dataset class
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]




In [7]:
model = torch.hub.load('pytorch/vision:v0.9.0', 'mobilenet_v3_large', pretrained=True)
# Freeze the weights of the model
for param in model.parameters():
    param.requires_grad = False

# Modify the last layer of the model to output 2 classes (male and female)
num_ftrs = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(num_ftrs, 2)

checkpoint = torch.load('/content/drive/MyDrive/MobileNetV3_Gender/model/model_12kv1_8.pt')
model.load_state_dict(checkpoint['model_state_dict'])
criterion = nn.CrossEntropyLoss()
criterion.load_state_dict(checkpoint['criterion_state_dict'])
optimizer = optim.Adam(model.classifier[-1].parameters(), lr=0.001)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

Downloading: "https://github.com/pytorch/vision/zipball/v0.9.0" to /root/.cache/torch/hub/v0.9.0.zip
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 68.6MB/s]


In [7]:
# Define the hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 4

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.33, random_state=42)
X_train = DataLoader(MyDataset(X_train), batch_size=batch_size, shuffle=False)
X_test = DataLoader(MyDataset(X_test), batch_size=batch_size, shuffle=False)
y_train = DataLoader(MyDataset(y_train), batch_size=batch_size, shuffle=False)
y_test = DataLoader(MyDataset(y_test), batch_size=batch_size, shuffle=False)


if False:
  # Load the pre-trained MobileNetV3 model
  model = torch.hub.load('pytorch/vision:v0.9.0', 'mobilenet_v3_large', pretrained=True)

  # Freeze the weights of the model
  for param in model.parameters():
      param.requires_grad = False

  # Modify the last layer of the model to output 2 classes (male and female)
  num_ftrs = model.classifier[-1].in_features
  model.classifier[-1] = nn.Linear(num_ftrs, 2)

  # Define the loss function and optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.classifier[-1].parameters(), lr=learning_rate)
else:
  model = torch.hub.load('pytorch/vision:v0.9.0', 'mobilenet_v3_large', pretrained=True)
  # Freeze the weights of the model
  for param in model.parameters():
      param.requires_grad = False

  # Modify the last layer of the model to output 2 classes (male and female)
  num_ftrs = model.classifier[-1].in_features
  model.classifier[-1] = nn.Linear(num_ftrs, 2)

  checkpoint = torch.load('/content/drive/MyDrive/MobileNetV3_Gender/model/model_6_end_kv2_24.pt')
  model.load_state_dict(checkpoint['model_state_dict'])
  criterion = nn.CrossEntropyLoss()
  criterion.load_state_dict(checkpoint['criterion_state_dict'])
  optimizer = optim.Adam(model.classifier[-1].parameters(), lr=learning_rate)
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# optimizer = optim.Adam(model.classifier[-1].parameters(), lr=learning_rate)
# Train the model
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, label in tqdm(zip(X_train,y_train)):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    
    # Print the training loss for each epoch
    epoch_loss = running_loss / len(y_train.dataset)
    print(f'Training Loss epoch {epoch+1}: {epoch_loss:.4f}')

# Evaluate the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, label in tqdm(zip(X_test,y_test)):
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()

# Print the accuracy of the model on the test set
test_acc = 100 * correct / total
print(f'Test Accuracy: {test_acc:.2f}%')


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0
69it [04:26,  3.87s/it]


Training Loss epoch 1: 0.4748


69it [04:24,  3.84s/it]


Training Loss epoch 2: 0.4409


69it [04:18,  3.74s/it]


Training Loss epoch 3: 0.4159


69it [04:14,  3.70s/it]


Training Loss epoch 4: 0.3936


34it [02:04,  3.67s/it]

Test Accuracy: 80.09%


In [15]:
# Evaluate the model on the test set
model = torch.hub.load('pytorch/vision:v0.9.0', 'mobilenet_v3_large', pretrained=True)
# Freeze the weights of the model
for param in model.parameters():
    param.requires_grad = False

# Modify the last layer of the model to output 2 classes (male and female)
num_ftrs = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(num_ftrs, 2)

checkpoint = torch.load('/content/drive/MyDrive/MobileNetV3_Gender/model/model_6_epoch_24.pt')
model.load_state_dict(checkpoint['model_state_dict'])
criterion = nn.CrossEntropyLoss()
criterion.load_state_dict(checkpoint['criterion_state_dict'])
optimizer = optim.Adam(model.classifier[-1].parameters(), lr=learning_rate)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
correct = 0
total = 0
with torch.no_grad():
    for inputs, label in tqdm(zip(dataset,labels)):
        outputs = model(inputs.unsqueeze(0))
        _, predicted = torch.max(outputs.data, 1)
        total += 1
        correct += (predicted == label).sum().item()

# Print the accuracy of the model on the test set
test_acc = 100 * correct / total
print(f'Test Accuracy: {test_acc:.2f}%')


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0
6500it [04:40, 23.18it/s]

Test Accuracy: 53.09%


In [9]:
batch_size = 64
learning_rate = 0.001
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.33, random_state=42)
X_train = DataLoader(MyDataset(X_train), batch_size=batch_size, shuffle=False)
X_test = DataLoader(MyDataset(X_test), batch_size=batch_size, shuffle=False)
y_train = DataLoader(MyDataset(y_train), batch_size=batch_size, shuffle=False)
y_test = DataLoader(MyDataset(y_test), batch_size=batch_size, shuffle=False)

In [3]:
from torchvision import models
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2) # binary classification (num_of_class == 2)
model.load_state_dict(torch.load("/content/drive/MyDrive/MobileNetV3_Gender/model/face_gender_classification_transfer_learning_with_ResNet18.pth"))

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [15]:
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'criterion_state_dict': criterion.state_dict(),
}, '/content/drive/MyDrive/MobileNetV3_Gender/model/model_epoch_28.pt')

Loss: 343.0775 Acc: 750.0000%
